In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime
from scipy.special import gamma, factorial
import scipy.special as sc

In [2]:
#定义变量 
thermal_power_station_strategy=np.zeros(shape=(4,1000)) 
wind_power_station_strategy=np.zeros(shape=(2,1000))
c_wind_speed= np.loadtxt("./data/c_wind_speed_uniform_distribution_(3,25).txt")      
kappa_wind_speed= np.loadtxt("./data/kappa_wind_speed_uniform_distribution_(2,3).txt")  
d_power_demand=np.loadtxt("./data/d_power_demand_normal_distribution_(70,5).txt") #时变的需求侧用电需求  阶梯式变化 
thermal_power_station_dual_variable=np.zeros(shape=(4,1000))
wind_power_station_dual_variable=np.zeros(shape=(2,1000))#两个风力发电站中有一个是受到拜占庭攻击的 假设第二个风力发电站是受到攻击的
E=np.loadtxt("6_Network_Topology_Double_Random_Adjacency_Matrix.txt")
thermal_power_station_cost=np.zeros(shape=(1,1000)) #火力发电成本
wind_power_station_cost=np.zeros(shape=(1,1000)) #风力发电成本
sum_cost=np.zeros(shape=(1,1000)) #总发电成本
#thermal_power_station_eta=[0.0375,0.0375,0.0625,0.035]
thermal_power_station_eta=[0.0675,0.0675,0.0925,0.0625]
thermal_power_station_zeta=[2,1.75,1,3]
thermal_power_station_xi=[0,0,0,0]
thermal_power_station_P_min=[50,20,15,10]
thermal_power_station_P_max=[200,120,80,100]
#wind_power_station_varphi=[3,6]
wind_power_station_varphi=[1,6]
wind_power_station_v_in=[3,5]
wind_power_station_v_out=[25,25]
wind_power_station_v_r=[13,15]
wind_power_station_sigma_ue=[5,5]
wind_power_station_sigma_oe=[30,20]
wind_power_station_P_r=[160,160]
wind_power_station_P_min=[0,0]
wind_power_station_P_max=[160,160]
sum_thermal_wind_power_station_strategy=np.zeros(shape=(1,1000))
step_size_primal=1
step_size_dual=1
tau_robust1=np.zeros(shape=(6,1000))#每个用户在每次迭代中均需要计算出一个tau值
regularization_parameter=0.001
small_Gaussian_Byzantine_attack=np.loadtxt("./data/small_Gaussian_Byzantine_attack_(-150,5).txt")

In [3]:
#火力，风力发电策略初始化
thermal_power_station_strategy[0][0]=0
thermal_power_station_strategy[1][0]=0
thermal_power_station_strategy[2][0]=0
thermal_power_station_strategy[3][0]=0
wind_power_station_strategy[0][0]=0
wind_power_station_strategy[1][0]=0
#初始总发电策略
sum_thermal_wind_power_station_strategy[0][0]=0
#初始化受到攻击的风力发电站的对偶变量
wind_power_station_dual_variable[1][0]=small_Gaussian_Byzantine_attack[0]

#安全发电站的初始总发电成本
#火力发电成本
for i in range(0,4):
    thermal_power_station_cost[0][0]=thermal_power_station_cost[0][0]+thermal_power_station_eta[i]*thermal_power_station_strategy[i][0]*thermal_power_station_strategy[i][0]+thermal_power_station_zeta[i]*thermal_power_station_strategy[i][0]
#风力发电成本
for i in range(0,1):
    wind_power_station_cost[0][0]=wind_power_station_cost[0][0]+wind_power_station_varphi[i]*wind_power_station_strategy[i][0]+wind_power_station_sigma_ue[i]*((wind_power_station_P_r[i]-wind_power_station_strategy[i][0])*(math.exp(-(math.pow(wind_power_station_v_r[i],kappa_wind_speed[0]))/(math.pow(c_wind_speed[0],kappa_wind_speed[0])))-math.exp(-(math.pow(wind_power_station_v_out[i],kappa_wind_speed[0]))/(math.pow(c_wind_speed[0],kappa_wind_speed[0]))))+(wind_power_station_P_r[i]*wind_power_station_v_in[i]/(wind_power_station_v_r[i]-wind_power_station_v_in[i])+wind_power_station_strategy[i][0])*(math.exp(-(math.pow(wind_power_station_v_r[i],kappa_wind_speed[0]))/(math.pow(c_wind_speed[0],kappa_wind_speed[0])))-math.exp(-(math.pow(wind_power_station_v_in[i]+((wind_power_station_v_r[i]-wind_power_station_v_in[i])*wind_power_station_strategy[i][0])/wind_power_station_P_r[i],kappa_wind_speed[0]))/(math.pow(c_wind_speed[0],kappa_wind_speed[0]))))+(wind_power_station_P_r[i]*c_wind_speed[0]/(wind_power_station_v_r[i]-wind_power_station_v_in[i]))*(sc.gammaincc(1+1/kappa_wind_speed[0],(math.pow(wind_power_station_v_in[i]+((wind_power_station_v_r[i]-wind_power_station_v_in[i])*wind_power_station_strategy[i][0])/wind_power_station_P_r[i],kappa_wind_speed[0]))/(math.pow(c_wind_speed[0],kappa_wind_speed[0])))*sc.gamma(1+1/kappa_wind_speed[0])-sc.gammaincc(1+1/kappa_wind_speed[0],math.pow(wind_power_station_v_r[i]/c_wind_speed[0],kappa_wind_speed[0]))*sc.gamma(1+1/kappa_wind_speed[0])))+wind_power_station_sigma_oe[i]*((wind_power_station_strategy[i][0])*(1-math.exp(-(math.pow(wind_power_station_v_in[i],kappa_wind_speed[0]))/(math.pow(c_wind_speed[0],kappa_wind_speed[0])))+math.exp(-(math.pow(wind_power_station_v_out[i],kappa_wind_speed[0]))/(math.pow(c_wind_speed[0],kappa_wind_speed[0]))))+(wind_power_station_P_r[i]*wind_power_station_v_in[i]/(wind_power_station_v_r[i]-wind_power_station_v_in[i])+wind_power_station_strategy[i][0])*(math.exp(-(math.pow(wind_power_station_v_in[i],kappa_wind_speed[0]))/(math.pow(c_wind_speed[0],kappa_wind_speed[0])))-math.exp(-(math.pow(wind_power_station_v_in[i]+((wind_power_station_v_r[i]-wind_power_station_v_in[i])*wind_power_station_strategy[i][0])/wind_power_station_P_r[i],kappa_wind_speed[0]))/(math.pow(c_wind_speed[0],kappa_wind_speed[0]))))+(wind_power_station_P_r[i]*c_wind_speed[0]/(wind_power_station_v_r[i]-wind_power_station_v_in[i]))*(sc.gammaincc(1+1/kappa_wind_speed[0],(math.pow(wind_power_station_v_in[i]+((wind_power_station_v_r[i]-wind_power_station_v_in[i])*wind_power_station_strategy[i][0])/wind_power_station_P_r[i],kappa_wind_speed[0]))/(math.pow(c_wind_speed[0],kappa_wind_speed[0])))*sc.gamma(1+1/kappa_wind_speed[0])-sc.gammaincc(1+1/kappa_wind_speed[0],math.pow(wind_power_station_v_in[i]/c_wind_speed[0],kappa_wind_speed[0]))*sc.gamma(1+1/kappa_wind_speed[0])))
#发电总成本
sum_cost[0][0]=thermal_power_station_cost[0][0]+wind_power_station_cost[0][0]

#计算每个发电站的邻居发电站
neighbor_robust1_help=[]
for i in range (0,6):
    for j in range (0,6):
        if E[i][j]!=0:
            neighbor_robust1_help.append(j)
        elif E[i][j]==0:
            neighbor_robust1_help.append(99)
neighbor_robust1=np.array(neighbor_robust1_help).reshape(6,6)

#计算安全的发电站的拜占庭邻居用户数量
q_robust1=[]
for i in range(0,6):
    q_robust1_help=0
    for j in range(0,6):
        if neighbor_robust1[i][j]==5:
            q_robust1_help=q_robust1_help+1
    q_robust1.append(q_robust1_help)
    
#计算出5个诚实用户的邻居中有多少是诚实用户 有多少是拜占庭用户
neighbor_honest_num_robust1=[]
neighbor_byzantine_num_robust1=[]
for i in range(0,6):
    h_num_robust1=0
    b_num_robust1=0
    u_neighbor_help_robust1=neighbor_robust1[i].tolist()#将用户i的邻居序号存在辅助变量u中
    if i==0 or i==3 or i==4 or i==5:
        u_neighbor_help_robust1 = list(set(u_neighbor_help_robust1))
        u_neighbor_help_robust1.remove(99)
    u_neighbor_help_robust1.remove(i)
    for j in range(0,len(u_neighbor_help_robust1)):
        if u_neighbor_help_robust1[j]==5:
            b_num_robust1=b_num_robust1+1
        else:
            h_num_robust1=h_num_robust1+1
    neighbor_honest_num_robust1.append(h_num_robust1)
    neighbor_byzantine_num_robust1.append(b_num_robust1)

In [4]:
def SCC_Byzantine_attack_decentralized_economic_dispatch(thermal_power_station_strategy,wind_power_station_strategy,c_wind_speed,kappa_wind_speed,d_power_demand,thermal_power_station_dual_variable,wind_power_station_dual_variable,E,thermal_power_station_cost,wind_power_station_cost,thermal_power_station_eta,thermal_power_station_zeta,thermal_power_station_P_min,thermal_power_station_P_max,wind_power_station_varphi,wind_power_station_sigma_ue,wind_power_station_sigma_oe,wind_power_station_P_min,wind_power_station_P_max,sum_thermal_wind_power_station_strategy,step_size_primal,step_size_dual):
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("***************************attack_free_online_decentralized_economic_dispatch*****************************")
    print("相关变量初始值为：")
    for i in range (0,4):
        print("第",(i+1),"个火力发电站的初始计划发电量为:",thermal_power_station_strategy[i][0])
    for i in range (0,2):
        print("第",(i+5),"个风力发电站的初始计划发电量为:",wind_power_station_strategy[i][0])
    print("****************************************************************************")
    for i in range (0,4):
        print("第",(i+1),"个火力发电站的初始对偶变量为:",thermal_power_station_dual_variable[i][0])
    for i in range (0,1):
        print("第",(i+5),"个风力发电站的初始对偶变量为:",wind_power_station_dual_variable[i][0])
    print("****************************************************************************")
    print("第",(6),"个受到Byzantin攻击的风力发电站的初始对偶变量为:",wind_power_station_dual_variable[1][0])
    print("****************************************************************************")
    print("安全发电站的初始总发电量:",sum_thermal_wind_power_station_strategy[0][0])
    print("*************************")
    print("需求侧用电需求:",d_power_demand[0])
    print("****************************************************************************")
    print("风速分布参数c:",c_wind_speed[0])
    print("****************************************************************************")
    print("风速分布参数kappa:",kappa_wind_speed[0])
    print("****************************************************************************")
    print("安全发电站的总发电成本：", sum_cost[0][0])
    print("****************************************************************************")
    print("原变量迭代步长为:",step_size_primal)
    print("*****************************************************************************")
    print("对偶变量迭代步长为:",step_size_dual)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,1000):
        #发电站更新计划发电量和对偶变量
        #对偶变量 先做梯度上升 再做聚合
        #安全发电站的梯度上升
        dual_variable_intermediate=[] #存放当前迭代中6个用户对对偶变量做完梯度下降后得到的中间值
        for i in range(0,4):
            dual_variable_intermediate.append(thermal_power_station_dual_variable[i][k-1]+step_size_dual*((1/6)*thermal_power_station_strategy[i][k-1]-(1/6)*d_power_demand[k-1]-regularization_parameter*thermal_power_station_dual_variable[i][k-1]))
        for j in range(0,1):
            dual_variable_intermediate.append(wind_power_station_dual_variable[j][k-1]+step_size_dual*((1/6)*wind_power_station_strategy[j][k-1]-(1/6)*d_power_demand[k-1]-regularization_parameter*wind_power_station_dual_variable[j][k-1]))
        #受到攻击的风力发电站的对偶值
        dual_variable_intermediate.append(wind_power_station_dual_variable[1][k-1])
        
        #SCC鲁棒聚合 安全的火力发电站的对偶更新
        for i in range (0,4):
            c_clipping_dual_robust1=[]#存放当前诚实用户对邻居的对偶变量裁剪后的值
            mix_sum_c_dual_robust1=0#存放clipping后的对偶值和
            u_neighbor_help_robust1=neighbor_robust1[i].tolist()#将用户i的邻居序号存在辅助变量u中
            if i==0 or i==3:
                u_neighbor_help_robust1 = list(set(u_neighbor_help_robust1))
                u_neighbor_help_robust1.remove(99)
            u_neighbor_help_robust1.remove(i)
            #计算用户i在当前阶段的tau值
            if neighbor_byzantine_num_robust1[i]==0:
                tau_robust1[i][k]=10000
            else:
                weight_sum_robust1=0
                for j in range(0,len(u_neighbor_help_robust1)):
                    if u_neighbor_help_robust1[j]!=5:
                        weight_sum_robust1=weight_sum_robust1+E[i][u_neighbor_help_robust1[j]]*(dual_variable_intermediate[u_neighbor_help_robust1[j]]-dual_variable_intermediate[i])*(dual_variable_intermediate[u_neighbor_help_robust1[j]]-dual_variable_intermediate[i])
                tau_robust1[i][k]=sqrt(weight_sum_robust1/(E[i][5]))                       
            #每个诚实用户首先对邻居传来的对偶变量进行剪裁clipping
            for j in range (0, len(u_neighbor_help_robust1)): #问题：分母会出现0的情况
                if (dual_variable_intermediate[u_neighbor_help_robust1[j]]-dual_variable_intermediate[i])==0:
                    c_clipping_dual_robust1.append(0)
                elif 1<=(tau_robust1[i][k]/(abs((dual_variable_intermediate[u_neighbor_help_robust1[j]]-dual_variable_intermediate[i])))):
                    c_clipping_dual_robust1.append(dual_variable_intermediate[u_neighbor_help_robust1[j]]-dual_variable_intermediate[i])
                else:
                    c_clipping_dual_robust1.append((dual_variable_intermediate[u_neighbor_help_robust1[j]]-dual_variable_intermediate[i])*((tau_robust1[i][k]/(abs((dual_variable_intermediate[u_neighbor_help_robust1[j]]-dual_variable_intermediate[i]))))))    
            #诚实用户聚合clipping后的邻居对偶值和自身的对偶值 
            for j in range(0,len(u_neighbor_help_robust1)):
                mix_sum_c_dual_robust1=mix_sum_c_dual_robust1+E[i][u_neighbor_help_robust1[j]]*(dual_variable_intermediate[i]+c_clipping_dual_robust1[j])
            #诚实用户基于SCC鲁棒聚合后的值进行自身的对偶变量的更新
            thermal_power_station_dual_variable[i][k]=E[i][i]*dual_variable_intermediate[i]+mix_sum_c_dual_robust1
        
        #聚合 安全的风力发电站的对偶更新 由于其邻居中没有拜占庭用户 所以直接按权重聚合即可
        for j in range(0,6):
            wind_power_station_dual_variable[0][k]=wind_power_station_dual_variable[0][k]+E[4][j]*dual_variable_intermediate[j]          

        #受到Byzantine攻击的风力发电站的对偶更新
        wind_power_station_dual_variable[1][k]=small_Gaussian_Byzantine_attack[k]           
        
        #安全的火力发电站计划发电量
        for i in range(0,4):
            thermal_power_station_strategy[i][k]=thermal_power_station_strategy[i][k-1]-step_size_primal*(2*thermal_power_station_eta[i]*thermal_power_station_strategy[i][k-1]+thermal_power_station_zeta[i]+(1/6)*thermal_power_station_dual_variable[i][k-1]) 
            if thermal_power_station_strategy[i][k]<=thermal_power_station_P_min[i]:
                thermal_power_station_strategy[i][k]=thermal_power_station_P_min[i]
            elif thermal_power_station_strategy[i][k]>=thermal_power_station_P_max[i]:
                thermal_power_station_strategy[i][k]=thermal_power_station_P_max[i]
        #安全的风力发电站计划发电量
        for i in range(0,1):
            wind_power_station_strategy[i][k]=wind_power_station_strategy[i][k-1]-step_size_primal*(wind_power_station_varphi[i]+wind_power_station_sigma_oe[i]+(wind_power_station_sigma_ue[i]+wind_power_station_sigma_oe[i])*math.exp(-(math.pow(wind_power_station_v_out[i],kappa_wind_speed[k-1]))/(math.pow(c_wind_speed[k-1],kappa_wind_speed[k-1])))+(-wind_power_station_sigma_ue[i]-wind_power_station_sigma_oe[i])*math.exp(-(math.pow((wind_power_station_v_in[i]+((wind_power_station_v_r[i]-wind_power_station_v_in[i])*wind_power_station_strategy[i][k-1])/wind_power_station_P_r[i]),kappa_wind_speed[k-1]))/(math.pow(c_wind_speed[k-1],kappa_wind_speed[k-1])))+(1/6)*wind_power_station_dual_variable[i][k-1])
            if wind_power_station_strategy[i][k]<=wind_power_station_P_min[i]:
                wind_power_station_strategy[i][k]=wind_power_station_P_min[i]
            elif wind_power_station_strategy[i][k]>=wind_power_station_P_max[i]:
                wind_power_station_strategy[i][k]=wind_power_station_P_max[i]
        #受到Byzantine攻击的风力发电站的计划发电量
        wind_power_station_strategy[1][k]=wind_power_station_strategy[1][k-1]
        
        #安全发电站的计算总计划发电
        #火力发电量
        for i in range(0,4):
            sum_thermal_wind_power_station_strategy[0][k]=sum_thermal_wind_power_station_strategy[0][k]+thermal_power_station_strategy[i][k]
        #风力发电量
        for i in range(0,1):
            sum_thermal_wind_power_station_strategy[0][k]=sum_thermal_wind_power_station_strategy[0][k]+wind_power_station_strategy[i][k]
        
        #安全发电站的计算总发电成本
        #火力发电成本
        for i in range(0,4):
            thermal_power_station_cost[0][k]=thermal_power_station_cost[0][k]+thermal_power_station_eta[i]*thermal_power_station_strategy[i][k]*thermal_power_station_strategy[i][k]+thermal_power_station_zeta[i]*thermal_power_station_strategy[i][k]
        #风力发电成本
        for i in range(0,1):
            wind_power_station_cost[0][k]=wind_power_station_cost[0][k]+wind_power_station_varphi[i]*wind_power_station_strategy[i][k]+wind_power_station_sigma_ue[i]*((wind_power_station_P_r[i]-wind_power_station_strategy[i][k])*(math.exp(-(math.pow(wind_power_station_v_r[i],kappa_wind_speed[k]))/(math.pow(c_wind_speed[k],kappa_wind_speed[k])))-math.exp(-(math.pow(wind_power_station_v_out[i],kappa_wind_speed[k]))/(math.pow(c_wind_speed[k],kappa_wind_speed[k]))))+(wind_power_station_P_r[i]*wind_power_station_v_in[i]/(wind_power_station_v_r[i]-wind_power_station_v_in[i])+wind_power_station_strategy[i][k])*(math.exp(-(math.pow(wind_power_station_v_r[i],kappa_wind_speed[k]))/(math.pow(c_wind_speed[k],kappa_wind_speed[k])))-math.exp(-(math.pow(wind_power_station_v_in[i]+((wind_power_station_v_r[i]-wind_power_station_v_in[i])*wind_power_station_strategy[i][k])/wind_power_station_P_r[i],kappa_wind_speed[k]))/(math.pow(c_wind_speed[k],kappa_wind_speed[k]))))+(wind_power_station_P_r[i]*c_wind_speed[k]/(wind_power_station_v_r[i]-wind_power_station_v_in[i]))*(sc.gammaincc(1+1/kappa_wind_speed[k],(math.pow(wind_power_station_v_in[i]+((wind_power_station_v_r[i]-wind_power_station_v_in[i])*wind_power_station_strategy[i][k])/wind_power_station_P_r[i],kappa_wind_speed[k]))/(math.pow(c_wind_speed[k],kappa_wind_speed[k])))*sc.gamma(1+1/kappa_wind_speed[k])-sc.gammaincc(1+1/kappa_wind_speed[k],math.pow(wind_power_station_v_r[i]/c_wind_speed[k],kappa_wind_speed[k]))*sc.gamma(1+1/kappa_wind_speed[k])))+wind_power_station_sigma_oe[i]*((wind_power_station_strategy[i][k])*(1-math.exp(-(math.pow(wind_power_station_v_in[i],kappa_wind_speed[k]))/(math.pow(c_wind_speed[k],kappa_wind_speed[k])))+math.exp(-(math.pow(wind_power_station_v_out[i],kappa_wind_speed[k]))/(math.pow(c_wind_speed[k],kappa_wind_speed[k]))))+(wind_power_station_P_r[i]*wind_power_station_v_in[i]/(wind_power_station_v_r[i]-wind_power_station_v_in[i])+wind_power_station_strategy[i][k])*(math.exp(-(math.pow(wind_power_station_v_in[i],kappa_wind_speed[k]))/(math.pow(c_wind_speed[k],kappa_wind_speed[k])))-math.exp(-(math.pow(wind_power_station_v_in[i]+((wind_power_station_v_r[i]-wind_power_station_v_in[i])*wind_power_station_strategy[i][k])/wind_power_station_P_r[i],kappa_wind_speed[k]))/(math.pow(c_wind_speed[k],kappa_wind_speed[k]))))+(wind_power_station_P_r[i]*c_wind_speed[k]/(wind_power_station_v_r[i]-wind_power_station_v_in[i]))*(sc.gammaincc(1+1/kappa_wind_speed[k],(math.pow(wind_power_station_v_in[i]+((wind_power_station_v_r[i]-wind_power_station_v_in[i])*wind_power_station_strategy[i][k])/wind_power_station_P_r[i],kappa_wind_speed[k]))/(math.pow(c_wind_speed[k],kappa_wind_speed[k])))*sc.gamma(1+1/kappa_wind_speed[k])-sc.gammaincc(1+1/kappa_wind_speed[k],math.pow(wind_power_station_v_in[i]/c_wind_speed[k],kappa_wind_speed[k]))*sc.gamma(1+1/kappa_wind_speed[k])))
        #发电总成本
        sum_cost[0][k]=thermal_power_station_cost[0][k]+wind_power_station_cost[0][k]
        
        #输出第k次迭代中的结果 每间隔10次迭代输出一次结果
        #if (k+1)%10==0:
        print(k,"次迭代中的结果：")
        for i in range (0,4):
            print("第",(i+1),"个火力发电站的计划发电量为:",thermal_power_station_strategy[i][k])
        for i in range (0,2):
            print("第",(i+5),"个风力发电站的计划发电量为:",wind_power_station_strategy[i][k])
        print("****************************************************************************")
        for i in range (0,4):
            print("第",(i+1),"个火力发电站的对偶变量为:",thermal_power_station_dual_variable[i][k])
        for i in range (0,1):
            print("第",(i+5),"个风力发电站的对偶变量为:",wind_power_station_dual_variable[i][k])
        print("****************************************************************************")
        print("第",(6),"个受到Byzantin攻击的风力发电站的初始对偶变量为:",wind_power_station_dual_variable[1][k])
        print("****************************************************************************")
        print("安全发电站的总发电量:",sum_thermal_wind_power_station_strategy[0][k])
        print("*************************")
        print("需求侧用电需求:",d_power_demand[k])
        print("****************************************************************************")
        print("风速分布参数c:",c_wind_speed[k])
        print("****************************************************************************")
        print("风速分布参数kappa:",kappa_wind_speed[k])
        print("****************************************************************************")        
        print("安全发电站的总发电成本：", sum_cost[0][k])
        print("****************************************************************************")
        print("原变量迭代步长为:",step_size_primal)
        print("****************************************************************************")
        print("对偶变量迭代步长为:",step_size_dual)
        print("****************************************************************************")

    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    return None  

In [5]:
SCC_Byzantine_attack_decentralized_economic_dispatch(thermal_power_station_strategy,wind_power_station_strategy,c_wind_speed,kappa_wind_speed,d_power_demand,thermal_power_station_dual_variable,wind_power_station_dual_variable,E,thermal_power_station_cost,wind_power_station_cost,thermal_power_station_eta,thermal_power_station_zeta,thermal_power_station_P_min,thermal_power_station_P_max,wind_power_station_varphi,wind_power_station_sigma_ue,wind_power_station_sigma_oe,wind_power_station_P_min,wind_power_station_P_max,sum_thermal_wind_power_station_strategy,step_size_primal,step_size_dual)

***************************attack_free_online_decentralized_economic_dispatch*****************************
相关变量初始值为：
第 1 个火力发电站的初始计划发电量为: 0.0
第 2 个火力发电站的初始计划发电量为: 0.0
第 3 个火力发电站的初始计划发电量为: 0.0
第 4 个火力发电站的初始计划发电量为: 0.0
第 5 个风力发电站的初始计划发电量为: 0.0
第 6 个风力发电站的初始计划发电量为: 0.0
****************************************************************************
第 1 个火力发电站的初始对偶变量为: 0.0
第 2 个火力发电站的初始对偶变量为: 0.0
第 3 个火力发电站的初始对偶变量为: 0.0
第 4 个火力发电站的初始对偶变量为: 0.0
第 5 个风力发电站的初始对偶变量为: 0.0
****************************************************************************
第 6 个受到Byzantin攻击的风力发电站的初始对偶变量为: -149.59297205
****************************************************************************
安全发电站的初始总发电量: 0.0
*************************
需求侧用电需求: 72.62781256
****************************************************************************
风速分布参数c: 8.301041
****************************************************************************
风速分布参数kappa: 2.66836324
**************************************************************************

对偶变量迭代步长为: 1
****************************************************************************
26 次迭代中的结果：
第 1 个火力发电站的计划发电量为: 90.20390803304511
第 2 个火力发电站的计划发电量为: 92.31857297618261
第 3 个火力发电站的计划发电量为: 73.46005552543893
第 4 个火力发电站的计划发电量为: 89.92461637874635
第 5 个风力发电站的计划发电量为: 73.38768464350177
第 6 个风力发电站的计划发电量为: 0.0
****************************************************************************
第 1 个火力发电站的对偶变量为: -78.2119973550831
第 2 个火力发电站的对偶变量为: -78.9607310232584
第 3 个火力发电站的对偶变量为: -79.55547274860045
第 4 个火力发电站的对偶变量为: -78.10135845208877
第 5 个风力发电站的对偶变量为: -78.50465856462776
****************************************************************************
第 6 个受到Byzantin攻击的风力发电站的初始对偶变量为: -155.52368054
****************************************************************************
安全发电站的总发电量: 419.29483755691473
*************************
需求侧用电需求: 76.57534104
****************************************************************************
风速分布参数c: 11.11125981
*****************************************************

风速分布参数kappa: 2.65936912
****************************************************************************
安全发电站的总发电成本： 2452.8155056917412
****************************************************************************
原变量迭代步长为: 1
****************************************************************************
对偶变量迭代步长为: 1
****************************************************************************
45 次迭代中的结果：
第 1 个火力发电站的计划发电量为: 68.41314681259114
第 2 个火力发电站的计划发电量为: 72.5080999611912
第 3 个火力发电站的计划发电量为: 56.87560208173312
第 4 个火力发电站的计划发电量为: 70.0969892840658
第 5 个风力发电站的计划发电量为: 78.78136326255431
第 6 个风力发电站的计划发电量为: 0.0
****************************************************************************
第 1 个火力发电站的对偶变量为: -65.73817132079876
第 2 个火力发电站的对偶变量为: -67.92901419236404
第 3 个火力发电站的对偶变量为: -69.0917370735931
第 4 个火力发电站的对偶变量为: -69.37662209446182
第 5 个风力发电站的对偶变量为: -65.63165204739009
****************************************************************************
第 6 个受到Byzantin攻击的风力发电站的初始对偶变量为: -145.69625775
********

第 3 个火力发电站的计划发电量为: 58.829227766336274
第 4 个火力发电站的计划发电量为: 73.19134756674251
第 5 个风力发电站的计划发电量为: 109.44310835600811
第 6 个风力发电站的计划发电量为: 0.0
****************************************************************************
第 1 个火力发电站的对偶变量为: -62.5757254837868
第 2 个火力发电站的对偶变量为: -66.51241276769231
第 3 个火力发电站的对偶变量为: -67.71310016702202
第 4 个火力发电站的对偶变量为: -68.62074057258734
第 5 个风力发电站的对偶变量为: -62.16492022755652
****************************************************************************
第 6 个受到Byzantin攻击的风力发电站的初始对偶变量为: -157.80483469
****************************************************************************
安全发电站的总发电量: 384.6200455909013
*************************
需求侧用电需求: 74.55388328
****************************************************************************
风速分布参数c: 19.39127935
****************************************************************************
风速分布参数kappa: 2.20652383
****************************************************************************
安全发电站的总发电成本： 3015.8698697033137
*****************

第 2 个火力发电站的对偶变量为: -76.0917559116728
第 3 个火力发电站的对偶变量为: -77.15786790764976
第 4 个火力发电站的对偶变量为: -77.16837304957066
第 5 个风力发电站的对偶变量为: -74.45027836294352
****************************************************************************
第 6 个受到Byzantin攻击的风力发电站的初始对偶变量为: -142.77966237
****************************************************************************
安全发电站的总发电量: 361.7836394637926
*************************
需求侧用电需求: 66.9462126
****************************************************************************
风速分布参数c: 16.49128078
****************************************************************************
风速分布参数kappa: 2.53722773
****************************************************************************
安全发电站的总发电成本： 2638.8929411385916
****************************************************************************
原变量迭代步长为: 1
****************************************************************************
对偶变量迭代步长为: 1
****************************************************************************
96 次迭代中的结果：


...

****************************************************************************
886 次迭代中的结果：
第 1 个火力发电站的计划发电量为: 75.05563199827215
第 2 个火力发电站的计划发电量为: 78.49220633913455
第 3 个火力发电站的计划发电量为: 62.61149362204454
第 4 个火力发电站的计划发电量为: 75.41333891763003
第 5 个风力发电站的计划发电量为: 78.14620153975667
第 6 个风力发电站的计划发电量为: 0.0
****************************************************************************
第 1 个火力发电站的对偶变量为: -71.87576297775337
第 2 个火力发电站的对偶变量为: -73.46317798187401
第 3 个火力发电站的对偶变量为: -74.54375080820876
第 4 个火力发电站的对偶变量为: -74.46570976667837
第 5 个风力发电站的对偶变量为: -71.74967088824887
****************************************************************************
第 6 个受到Byzantin攻击的风力发电站的初始对偶变量为: -154.85015789
****************************************************************************
安全发电站的总发电量: 369.71887241683794
*************************
需求侧用电需求: 68.19035828
****************************************************************************
风速分布参数c: 9.99529554
****************************************************************

905 次迭代中的结果：
第 1 个火力发电站的计划发电量为: 68.39310174476651
第 2 个火力发电站的计划发电量为: 74.1838756976845
第 3 个火力发电站的计划发电量为: 58.70554938429942
第 4 个火力发电站的计划发电量为: 72.54625413268005
第 5 个风力发电站的计划发电量为: 82.04969848005702
第 6 个风力发电站的计划发电量为: 0.0
****************************************************************************
第 1 个火力发电站的对偶变量为: -66.80498862916788
第 2 个火力发电站的对偶变量为: -68.85255770621434
第 3 个火力发电站的对偶变量为: -69.98063740058672
第 4 个火力发电站的对偶变量为: -70.21373768372808
第 5 个风力发电站的对偶变量为: -66.70230689368935
****************************************************************************
第 6 个受到Byzantin攻击的风力发电站的初始对偶变量为: -143.6338674
****************************************************************************
安全发电站的总发电量: 355.8784794394875
*************************
需求侧用电需求: 65.26473893
****************************************************************************
风速分布参数c: 10.98816331
****************************************************************************
风速分布参数kappa: 2.95947097
******************************************

第 1 个火力发电站的计划发电量为: 72.46033547302761
第 2 个火力发电站的计划发电量为: 76.15045224615342
第 3 个火力发电站的计划发电量为: 60.688204351687865
第 4 个火力发电站的计划发电量为: 73.18275530546666
第 5 个风力发电站的计划发电量为: 60.139478658036566
第 6 个风力发电站的计划发电量为: 0.0
****************************************************************************
第 1 个火力发电站的对偶变量为: -74.22578369515077
第 2 个火力发电站的对偶变量为: -74.99965916176295
第 3 个火力发电站的对偶变量为: -75.88965094313309
第 4 个火力发电站的对偶变量为: -75.26370800081637
第 5 个风力发电站的对偶变量为: -74.21297223202653
****************************************************************************
第 6 个受到Byzantin攻击的风力发电站的初始对偶变量为: -150.945037
****************************************************************************
安全发电站的总发电量: 342.6212260343721
*************************
需求侧用电需求: 70.88564198
****************************************************************************
风速分布参数c: 8.32011436
****************************************************************************
风速分布参数kappa: 2.38448273
******************************************************

956 次迭代中的结果：
第 1 个火力发电站的计划发电量为: 74.5982050403939
第 2 个火力发电站的计划发电量为: 78.64522249582191
第 3 个火力发电站的计划发电量为: 62.347528846370224
第 4 个火力发电站的计划发电量为: 76.36604255417978
第 5 个风力发电站的计划发电量为: 90.42336247970597
第 6 个风力发电站的计划发电量为: 0.0
****************************************************************************
第 1 个火力发电站的对偶变量为: -71.98480169556979
第 2 个火力发电站的对偶变量为: -73.29491226910508
第 3 个火力发电站的对偶变量为: -74.38875669398045
第 4 个火力发电站的对偶变量为: -74.00413011136747
第 5 个风力发电站的对偶变量为: -71.86110137815791
****************************************************************************
第 6 个受到Byzantin攻击的风力发电站的初始对偶变量为: -149.44545166
****************************************************************************
安全发电站的总发电量: 382.3803614164718
*************************
需求侧用电需求: 72.52881724
****************************************************************************
风速分布参数c: 5.85455319
****************************************************************************
风速分布参数kappa: 2.1109125
******************************************

第 4 个火力发电站的计划发电量为: 78.27025652809749
第 5 个风力发电站的计划发电量为: 78.3136769112897
第 6 个风力发电站的计划发电量为: 0.0
****************************************************************************
第 1 个火力发电站的对偶变量为: -78.32444700932942
第 2 个火力发电站的对偶变量为: -78.94785101770945
第 3 个火力发电站的对偶变量为: -79.76823154014883
第 4 个火力发电站的对偶变量为: -78.82931553877623
第 5 个风力发电站的对偶变量为: -78.3990288059968
****************************************************************************
第 6 个受到Byzantin攻击的风力发电站的初始对偶变量为: -154.83293851
****************************************************************************
安全发电站的总发电量: 385.3541538317961
*************************
需求侧用电需求: 69.49133351
****************************************************************************
风速分布参数c: 11.82837734
****************************************************************************
风速分布参数kappa: 2.5563993
****************************************************************************
安全发电站的总发电成本： 2895.4369842956494
**********************************************************

In [6]:
#数据存储
np.savetxt('./data/04_SCC_optimal_parameters_small_Gaussian_Byzantine_attack_sum_cost.txt',sum_cost,fmt='%0.8f')
np.savetxt('./data/04_SCC_optimal_parameters_small_Gaussian_Byzantine_attack_sum_thermal_wind_power_station_strategy.txt',sum_thermal_wind_power_station_strategy,fmt='%0.8f')

In [7]:
#评价指标：动态regret和累计的约束违反

In [8]:
#动态regret

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime
from scipy.special import gamma, factorial
import scipy.special as sc

In [10]:
sum_cost=np.loadtxt("./data/04_SCC_optimal_parameters_small_Gaussian_Byzantine_attack_sum_cost.txt")
optimal_sum_cost=np.loadtxt("./data/00_optimal_sum_cost_varying_D_c_kappa_normal_distribution_(70,5).txt")

In [11]:
dynamic_regret=[]
dynamic_regret_T=[]
for T in range(1, 1000):
    dynamic_regret_auxiliary_variable=0
    for t in range(0, T):
        dynamic_regret_auxiliary_variable=dynamic_regret_auxiliary_variable+sum_cost[t+1]-optimal_sum_cost[t+1]
    dynamic_regret.append(dynamic_regret_auxiliary_variable)
    dynamic_regret_T.append((dynamic_regret_auxiliary_variable)/T)

In [12]:
dynamic_regret_T

[-1203.6743223,
 -1458.1681773,
 -1589.6554169233332,
 -1522.617072225,
 -1545.8889986059999,
 -1491.939234195,
 -1485.42125273,
 -1425.360851405,
 -1353.2735766899998,
 -1399.4079975089999,
 -1355.4010644518182,
 -1245.5804185258332,
 -1167.3196382576923,
 -1077.4598792771428,
 -956.0454241133333,
 -858.83164701125,
 -755.0622533,
 -625.6876906955555,
 -521.0431544384211,
 -432.72639736250005,
 -356.6926567433334,
 -251.21112756545455,
 -193.3554794373913,
 -126.49630506458332,
 -64.24080750039998,
 -38.48366890999999,
 -20.031231276666656,
 14.42711942285715,
 42.228218280689646,
 78.27523323899999,
 104.0453115583871,
 113.5546229178125,
 125.57786420090909,
 136.37775512352943,
 168.64303131600002,
 180.15386230638893,
 183.89837376972974,
 209.8646874489474,
 223.40381116153853,
 234.06471137575008,
 237.32497717146353,
 231.99518975500013,
 249.90840105651174,
 247.95535800363643,
 247.3437099533334,
 248.77075710478263,
 266.02121696042553,
 263.00496188666665,
 256.475522791428

In [13]:
#绘图 dynamic_regret和dynamic_regret_T

In [14]:
sub1=plt.subplot(1, 2, 1)##############################################################
iteration=[]
for i in range(0,999):
    iteration.append(i)

#在同一幅图片上画3条折线
A,=plt.plot(iteration,dynamic_regret,color = 'red',linewidth = '1.5',label='attack-free online primal-dual algorithm')


#设置图例并且设置图例的字体及大小
font1 = {
'size'   : 18,
}
#legend = plt.legend(handles=[A],prop=font1)
 
#设置坐标刻度值的大小以及刻度值的字体
plt.tick_params(labelsize=18)
 
#设置横纵坐标的名称以及对应字体格式
font2 = {
'size'   : 18,
}
plt.xlabel('T',font2)
plt.ylabel(r'Dynamic regret $Reg(T)$',font2)
plt.title('varying $D_{t}$',font2) 

sub2=plt.subplot(1, 2, 2)##############################################################
iteration=[]
for i in range(0,999):
    iteration.append(i)

#在同一幅图片上画3条折线
A,=plt.plot(iteration,dynamic_regret_T,color = 'red',linewidth = '1.5',label='attack-free online primal-dual algorithm')


#设置图例并且设置图例的字体及大小
font1 = {
'size'   : 18,
}
#legend = plt.legend(handles=[A],prop=font1)
 
#设置坐标刻度值的大小以及刻度值的字体
plt.tick_params(labelsize=18)
 
#设置横纵坐标的名称以及对应字体格式
font2 = {
'size'   : 18,
}
plt.xlabel('T',font2)
plt.ylabel(r'Dynamic regret $Reg(T)/T$',font2)
plt.title('varying $D_{t}$',font2) 

plt.legend(handles=[A],prop={'size':18},loc="upper center", bbox_to_anchor=[-0.3, -0.2],
           ncol=18, shadow=True, fancybox=True)

plt.subplots_adjust(wspace=0.5,hspace=0.5) 
plt.gcf().set_size_inches((15, 5))

#plt.savefig('./figures/01-figure1_dynamic_regret_attack_free_varying_D.pdf',format='pdf', bbox_inches='tight')
#plt.savefig('./figures/01-figure1_dynamic_regret_attack_free_varying_D.png',format='png', bbox_inches='tight')

In [15]:
#累计的约束违反

In [16]:
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime
from scipy.special import gamma, factorial
import scipy.special as sc

In [17]:
sum_thermal_wind_power_station_strategy=np.loadtxt("./data/04_SCC_optimal_parameters_small_Gaussian_Byzantine_attack_sum_thermal_wind_power_station_strategy.txt")
d_power_demand=np.loadtxt("./data/d_power_demand_normal_distribution_(70,5).txt")

In [18]:
accumulative_constraint_violation=[]
accumulative_constraint_violation_T=[]
for T in range(1, 1000):
    accumulative_constraint_violation_auxiliary_variable=0
    for t in range(0, T): 
        accumulative_constraint_violation_auxiliary_variable=accumulative_constraint_violation_auxiliary_variable+(1/5)*sum_thermal_wind_power_station_strategy[t+1]-d_power_demand[t+1]
    accumulative_constraint_violation.append(abs(accumulative_constraint_violation_auxiliary_variable))
    accumulative_constraint_violation_T.append(abs(accumulative_constraint_violation_auxiliary_variable)/T)

In [19]:
accumulative_constraint_violation_T

[44.659041904000006,
 48.588688235000006,
 50.17187312533334,
 50.5269115015,
 49.8808214904,
 48.762567708333336,
 47.48856498800001,
 44.845898918,
 42.74765415777778,
 42.0206582558,
 40.42686813072728,
 37.47903203933334,
 35.0290672903077,
 32.72693842542858,
 29.863709972400002,
 27.391548011250006,
 25.35176851058824,
 23.378914212,
 20.979872097052635,
 19.181498298900003,
 17.597070116380955,
 16.05391464236364,
 14.7347370766087,
 13.707979832166671,
 12.292383573520006,
 11.539460110230776,
 10.93255449355556,
 10.041034875357147,
 9.372323616689659,
 8.57746914626667,
 7.8398430278709705,
 7.619248834125003,
 7.161101021818185,
 6.764097215882356,
 6.197343925200003,
 5.809502785777781,
 5.517002615783787,
 4.800554949315793,
 4.432285581641029,
 4.250604215250003,
 4.049932857073174,
 4.114600618333337,
 4.018827266186049,
 3.865629744727275,
 3.749694720177781,
 3.5884500011739164,
 3.5315932120851095,
 3.4434185200416696,
 3.504697105428574,
 3.5094770283600023,
 3.48754

In [20]:
#绘图 accumulative_constraint_violation和accumulative_constraint_violation_T

In [21]:
sub1=plt.subplot(1, 2, 1)##############################################################
iteration=[]
for i in range(0,999):
    iteration.append(i)

#在同一幅图片上画3条折线
A,=plt.plot(iteration,accumulative_constraint_violation,color = 'green',linewidth = '1.5',label='attack-free online primal-dual algorithm')


#设置图例并且设置图例的字体及大小
font1 = {
'size'   : 18,
}
#legend = plt.legend(handles=[A],prop=font1)
 
#设置坐标刻度值的大小以及刻度值的字体
plt.tick_params(labelsize=18)
 
#设置横纵坐标的名称以及对应字体格式
font2 = {
'size'   : 18,
}
plt.xlabel('T',font2)
plt.ylabel(r'Constraint violation $Fit(T)$',font2)
plt.title('varying $D_{t}$',font2) 

sub2=plt.subplot(1, 2, 2)##############################################################
iteration=[]
for i in range(0,999):
    iteration.append(i)

#在同一幅图片上画3条折线
A,=plt.plot(iteration,accumulative_constraint_violation_T,color = 'green',linewidth = '1.5',label='attack-free online primal-dual algorithm')


#设置图例并且设置图例的字体及大小
font1 = {
'size'   : 18,
}
#legend = plt.legend(handles=[A],prop=font1)
 
#设置坐标刻度值的大小以及刻度值的字体
plt.tick_params(labelsize=18)
 
#设置横纵坐标的名称以及对应字体格式
font2 = {
'size'   : 18,
}
plt.xlabel('T',font2)
plt.ylabel(r'Constraint violation $Fit(T)/T$',font2)
plt.title('varying $D_{t}$',font2) 

plt.legend(handles=[A],prop={'size':18},loc="upper center", bbox_to_anchor=[-0.3, -0.2],
           ncol=18, shadow=True, fancybox=True)

plt.subplots_adjust(wspace=0.5,hspace=0.5) 
plt.gcf().set_size_inches((15, 5))


#plt.savefig('./figures/01-figure2_constraint_violation_attack_free_varying_D.pdf',format='pdf', bbox_inches='tight')
#plt.savefig('./figures/01-figure2_constraint_violation_attack_free_varying_D.png',format='png', bbox_inches='tight')